In [4]:
from langchain.document_loaders import UnstructuredEPubLoader, DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.embeddings.openai import OpenAIEmbeddings


from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma, Pinecone, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain


from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [5]:
loader = DirectoryLoader('./resources/ngnL/', glob="**/*.epub")

In [6]:
data = loader.load()
print(f'documents: {len(data)}')

documents: 11


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap = 25
)

In [8]:
split_docs = text_splitter.split_documents(data)
print(f'documents:{len(split_docs)}')

documents:6001


In [9]:
embeddings = OpenAIEmbeddings()

In [10]:
database = FAISS.from_documents(split_docs, embeddings)

In [11]:
database.save_local("./db/")

In [12]:
database.load_local("./db/", embeddings=embeddings)

In [13]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=database,return_source_documents=True)

/opt/homebrew/Caskroom/miniconda/base/envs/LangChainLabs/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:201: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [15]:
result = qa({"query": "阿兹莉尔现在怎么样了？"})
print(result)

{'query': '阿兹莉尔现在怎么样了？', 'result': ' 阿兹莉尔正凝视着映在虚空中的影像，仍在思索。名为绝望的黑暗中出现一道光明，朝着阿兹莉尔走过来的，名符其实的天使小吉正在和她说话。', 'source_documents': [Document(page_content='是啊！现在正是攸关艾尔奇亚与全人类种生死的关键时刻。\n\n只要想到这样的状况，根本不可能会心动的！\n\n我要让心灵归于虚无，撑过这两个小时。\n\n为了拯救艾尔奇亚，这是现在的我唯一能采取的行动！！\n\n史蒂芙如此告诉自己，怀着钢铁一般坚定的决心。但是──\n\n「……呼～❤」\n\n「呀啊啊嗯！？」\n\n突然，有人从背后朝着她的耳朵吹了一口气，瞬间粉碎了她的决心。\n\n「你、你这是在做什么！？吉普莉尔！」\n\n史蒂芙转头责问突然出现在背后的吉普莉尔。\n\n吉普莉尔却是一点歉意都没有，只是一脸不解。', metadata={'source': 'resources/ngnL/11.epub'}), Document(page_content='■■■\n\n「……看来也有其他人懂了……这样你还要说只有我是特别的吗？」\n\n吉普莉尔苦笑着说道。\n\n阿兹莉尔面无表情，但是在那张面具之下则是苦恼扭曲的模样。\n\n──那是什么？到底是怎么一回事？完全看不懂。\n\n凝视着映在虚空中的影像，阿兹莉尔仍在思索。\n\n在逐渐崩毁的阿邦特•赫伊姆之中，飞翔的两人出神入化地回避障碍物。\n\n他们笔直地──往这里前进，彷佛知道自己的位置一般──！', metadata={'source': 'resources/ngnL/05.epub'}), Document(page_content='她是天翼种的全权代理──『第一号个体』阿兹莉尔。\n\n只不过史蒂芙之所以会倒抽一口气，不是因为『她的模样』。\n\n而是从城堡窗户望见的天空──就像被盖住似地封闭了。\n\n她大概将阿邦特•赫伊姆整个转移至艾尔奇亚上空了。\n\n「可不是人人都像小吉那样通情达理哦──铁屑？」\n\n她的双眸之中，散发出非比寻常的──『敌意』。\n\n与吉普莉尔和机凯种交锋时相比，『情感』性质完全不同。', metadata={'source': 'resources/ngnL/09.epub'})